# Вебинар 2. Бейзлайны и детерминированные алгоритмы item-item

In [93]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [94]:
data = pd.read_csv('./data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [95]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [96]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    return np.random.choice(items_weights['item_id'], size=n, replace=False, p=items_weights['weight']).tolist()

Сначала подготовим датафрейм с весами. Т.к. логарифм от числа меньшего 1 является отрицательным значением, то прибавим 1 ко всем значениям. Это позволит избавиться от отрицательных значений, но при этом не сильно повлияет на соотношение весов.

In [97]:
def get_log_items_weights(data, field_item_id, field_item_weight, field_log_weight):
    """
    Логарифмирование весов
    
    Input
    -----
    data - Логарифмируемый датафрейм
    
    field_item_id - Название поля с идентификатором объекта
    
    field_item_weight - Название исходного логарифмируемого поля
    
    field_log_weight - Название результирующего логарифмируемого поля

    Output
    -----
    Датафрейм с идентификатором объекта и логарифмированным весом
    """    
    
    items_weights = data[[field_item_id, field_item_weight]].groupby(field_item_id)[field_item_weight].sum().reset_index()
    log = np.log(items_weights[field_item_weight] + 1)
    items_weights[field_log_weight] = log / np.sum(log)
    return items_weights[[field_item_id, field_log_weight]]

Создадим датафрейм с покупками юзеров

In [98]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(10)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8..."
6,13,"[6534178, 1104146, 829197, 840361, 862070, 884..."
7,14,"[840601, 867293, 933067, 951590, 952408, 96569..."
8,15,"[910439, 1082185, 959076, 1023958, 1082310, 13..."
9,16,"[1062973, 1082185, 13007710]"


Сделаем предсказания

In [99]:
%%time

log_weights = get_log_items_weights(data_train, 'item_id', 'sales_value', 'weight')
result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(log_weights, n=5))
result.head(2)

Wall time: 1.48 s


,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[872324, 9420354, 9706822, 9858988, 6534703]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1015731, 12604869, 13910839, 1042643, 869728]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

csv коряво сохраняет списки, поэтому пересохранил результат в pickle

In [100]:
#result = pd.read_csv('./predictions/predictions_basic.csv')
predictions = pd.read_pickle('./predictions/predictions_basic.pkl')
predictions.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[13115497, 1023761, 1086038, 1026867, 879546]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[9575072, 2021784, 13189514, 822768, 643665]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]"


Добавим взвешанный алгоритм

In [101]:
%%time

log_weights = get_log_items_weights(data_train, 'item_id', 'sales_value', 'weight')
predictions['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(log_weights, n=5))
predictions.head(2)

Wall time: 1.47 s


,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[13115497, 1023761, 1086038, 1026867, 879546]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]","[1022668, 10121617, 1647059, 1057743, 12263792]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[9575072, 2021784, 13189514, 822768, 643665]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]","[977330, 12386240, 10201699, 1005961, 9575176]"


Функция из вебинара 1

In [102]:
def precision_at_k(recommended_list, bought_list, k=5):
    recommended_list = recommended_list[:k]
    return np.isin(bought_list, recommended_list).sum() / len(recommended_list)

Посчитаем метрику Precision@5 для каждого алгоритма

In [103]:
result_at_k = []
calc_fields = ['random_recommendation', 'popular_recommendation', 'itemitem', 'cosine', 'tfidf', 
               'own_purchases', 'weighted_random_recommendation']
for c in calc_fields:
    at_k = predictions.apply(lambda x: precision_at_k(x[c], x['actual'], k=5), axis=1).mean()
    result_at_k.append([c, at_k])
    
result_at_k = pd.DataFrame(result_at_k, columns={'algorithm', 'precision_at_k'}).set_index('algorithm')
result_at_k

,precision_at_k
algorithm,
random_recommendation,0.000392
popular_recommendation,0.155240
itemitem,0.136925
cosine,0.132909
tfidf,0.138981
own_purchases,0.179693
weighted_random_recommendation,0.000784


Лучший результат показывает алгоритм основанный на собственных покупках.

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.
- Попробуйте стратегии ансамблирования изученных алгоритмов

### Попробуем улучшить бейзлайны

In [104]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


Найдём топ-5000 товаров

In [105]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
top5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()
top5000[:5]

[6534178, 6533889, 6534166, 6544236, 1404121]

In [106]:
data_top5000 = data_train[np.isin(data_train['item_id'], top5000)]

#### Random recommendation

In [107]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    return np.random.choice(items, size=n, replace=False).tolist()

In [108]:
%%time

items = data_top5000.item_id.unique()
result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))
result.head(2)

Wall time: 200 ms


,user_id,actual,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1078011, 1051468, 884518, 1029479, 6544236]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[893651, 1068957, 1003600, 868727, 1048746]"


In [109]:
result_at_k.at['random_recommendation', 'optimized'] = \
result.apply(lambda x: precision_at_k(x['random_recommendation'], x['actual']), axis=1).mean()

#### Popularity-based recommendation

In [110]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""

    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    return popular.head(n).item_id.tolist()

In [111]:
%%time

popular_recs = popularity_recommendation(data_top5000, n=5)
result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)
result.head(2)

Wall time: 46.9 ms


,user_id,actual,random_recommendation,popular_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1078011, 1051468, 884518, 1029479, 6544236]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[893651, 1068957, 1003600, 868727, 1048746]","[6534178, 6533889, 1029743, 6534166, 1082185]"


In [112]:
result_at_k.at['popular_recommendation', 'optimized'] = \
result.apply(lambda x: precision_at_k(x['popular_recommendation'], x['actual']), axis=1).mean()

#### Weighted random recommender

In [113]:
%%time

log_weights = get_log_items_weights(data_top5000, 'item_id', 'sales_value', 'weight')
result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(log_weights, n=5))
result.head(2)

Wall time: 267 ms


,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1078011, 1051468, 884518, 1029479, 6544236]","[6534178, 6533889, 1029743, 6534166, 1082185]","[960318, 1039831, 843898, 12524323, 1090701]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[893651, 1068957, 1003600, 868727, 1048746]","[6534178, 6533889, 1029743, 6534166, 1082185]","[898212, 958569, 1016276, 1108152, 1109805]"


In [114]:
result_at_k.at['weighted_random_recommendation', 'optimized'] = \
result.apply(lambda x: precision_at_k(x['weighted_random_recommendation'], x['actual']), axis=1).mean()

Сравним исходные и новые расчёты

In [115]:
result_at_k

,precision_at_k,optimized
algorithm,,
random_recommendation,0.000392,0.006170
popular_recommendation,0.155240,0.155240
itemitem,0.136925,NaN
cosine,0.132909,NaN
tfidf,0.138981,NaN
own_purchases,0.179693,NaN
weighted_random_recommendation,0.000784,0.006464


Алгоритмы действительно получилось улучшить, за исключением popular_recommendation, т.к. он и до этого был основан на алгоритме топ популярных товаров

### Попробуем улучшить разные варианты ItemItemRecommender

#### ItemItemRecommender

In [116]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

C:\Users\rseme\.conda\envs\tf2-gpu\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [117]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

Попробуем подобрать k перебором

In [118]:
%%time

at_k_list = []

for k in range(1, 10):
    model = ItemItemRecommender(K=k, num_threads=4) # K - кол-во билжайших соседей
    model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
              show_progress=True)

    result['itemitem'] = result['user_id'].\
        apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                        model.recommend(userid=userid_to_id[x], 
                                        user_items=sparse_user_item,   # на вход user-item matrix
                                        N=5, 
                                        filter_already_liked_items=False, 
                                        filter_items=None, 
                                        recalculate_user=True)])
    
    at_k = result.apply(lambda x: precision_at_k(x['itemitem'], x['actual']), axis=1).mean()
    at_k_list.append(at_k)
    print(f'k = {k}, at_k = {at_k}')


k = 1, at_k = 0.17998694090760692



k = 2, at_k = 0.192009467841985



k = 3, at_k = 0.1860920666013712



k = 4, at_k = 0.14495592556317335



k = 5, at_k = 0.13692458374143



k = 6, at_k = 0.14201762977473065



k = 7, at_k = 0.14485798237022526



k = 8, at_k = 0.14720861900097945



k = 9, at_k = 0.14848188050930464
Wall time: 4.67 s


Лучшая модель получилась при k=2. Добавим этот результат в итоговую таблицу

In [119]:
result_at_k.at['itemitem', 'optimized'] = at_k_list[1]

#### CosineRecommender

In [120]:
%%time

at_k_list = []

for k in range(1, 10):
    model = CosineRecommender(K=k, num_threads=4) # K - кол-во билжайших соседей
    model.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=True)

    result['cosine'] = result['user_id'].\
        apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                        model.recommend(userid=userid_to_id[x], 
                                        user_items=sparse_user_item,   # на вход user-item matrix
                                        N=5, 
                                        filter_already_liked_items=False, 
                                        filter_items=None, 
                                        recalculate_user=True)])
    
    at_k = result.apply(lambda x: precision_at_k(x['cosine'], x['actual']), axis=1).mean()
    at_k_list.append(at_k)
    print(f'k = {k}, at_k = {at_k}')


k = 1, at_k = 0.17276363042768528



k = 2, at_k = 0.14977962781586682



k = 3, at_k = 0.1427032321253673



k = 4, at_k = 0.13633692458374141



k = 5, at_k = 0.1329089128305583



k = 6, at_k = 0.13388834476003916



k = 7, at_k = 0.13134182174338885



k = 8, at_k = 0.1306562193927522



k = 9, at_k = 0.13114593535749267
Wall time: 5 s


Лучшая модель получилась при k=1. Добавим этот результат в итоговую таблицу

In [121]:
result_at_k.at['cosine', 'optimized'] = at_k_list[0]

#### TFIDFRecommender

In [122]:
%%time

at_k_list = []

for k in range(1, 10):
    model = TFIDFRecommender(K=k, num_threads=4) # K - кол-во билжайших соседей
    model.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=True)

    result['tfidf'] = result['user_id'].\
        apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                        model.recommend(userid=userid_to_id[x], 
                                        user_items=sparse_user_item,   # на вход user-item matrix
                                        N=5, 
                                        filter_already_liked_items=False, 
                                        filter_items=None, 
                                        recalculate_user=False)])

    at_k = result.apply(lambda x: precision_at_k(x['tfidf'], x['actual']), axis=1).mean()
    at_k_list.append(at_k)
    print(f'k = {k}, at_k = {at_k}')


k = 1, at_k = 0.1290809663728371



k = 2, at_k = 0.1559500489715965



k = 3, at_k = 0.14642507345739472



k = 4, at_k = 0.14084231145935358



k = 5, at_k = 0.13898139079333988



k = 6, at_k = 0.138001958863859



k = 7, at_k = 0.13937316356513224



k = 8, at_k = 0.13819784524975517



k = 9, at_k = 0.1394711067580803
Wall time: 5.24 s


Лучшая модель получилась при k=2. Добавим этот результат в итоговую таблицу

In [123]:
result_at_k.at['tfidf', 'optimized'] = at_k_list[1]

#### own_purchases

In [125]:
%%time

at_k_list = []

for k in range(1, 10):
    model = ItemItemRecommender(K=k, num_threads=4) # K - кол-во билжайших соседей
    model.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=True)

    result['own_purchases'] = result['user_id'].\
        apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                        model.recommend(userid=userid_to_id[x], 
                                        user_items=sparse_user_item,   # на вход user-item matrix
                                        N=5, 
                                        filter_already_liked_items=False, 
                                        filter_items=[999999], 
                                        recalculate_user=False)])
    
    at_k = result.apply(lambda x: precision_at_k(x['own_purchases'], x['actual']), axis=1).mean()
    at_k_list.append(at_k)
    print(f'k = {k}, at_k = {at_k}')


k = 1, at_k = 0.17969311132876264



k = 2, at_k = 0.192009467841985



k = 3, at_k = 0.1860920666013712



k = 4, at_k = 0.14495592556317335



k = 5, at_k = 0.13692458374143



k = 6, at_k = 0.14201762977473065



k = 7, at_k = 0.14485798237022526



k = 8, at_k = 0.14720861900097945



k = 9, at_k = 0.14848188050930464
Wall time: 4.64 s


Лучшая модель получилась при k=2. Добавим этот результат в итоговую таблицу

In [126]:
result_at_k.at['own_purchases', 'optimized'] = at_k_list[1]

В результате получили следующую таблицу с исходными и оптимизированными значениями метрики

In [127]:
result_at_k

,precision_at_k,optimized
algorithm,,
random_recommendation,0.000392,0.006170
popular_recommendation,0.155240,0.155240
itemitem,0.136925,0.192009
cosine,0.132909,0.172764
tfidf,0.138981,0.155950
own_purchases,0.179693,0.192009
weighted_random_recommendation,0.000784,0.006464
